**Boost Ideal CCM**

Dados los datos: 
- $F$<sub>sw</sub>
- $V$<sub>d</sub>
- $V$<sub>o</sub>
- $\Delta$ $V$<sub>o</sub>$/V$<sub>o</sub>
- $V$<sub>GG</sub>

Obtenemos:

In [4]:
import numpy as np

#Data
Fsw = 50E3
Ts = 1/Fsw
Vd = 6
Vo = 15
Vo_tol = 0.07
Delta_Vo = Vo*Vo_tol
VGG = 18

L = 220E-6
L_tol = 0.1
L_Q = 20

#Definir una R de carga
R = 1500

#Calculamos D inicial y vemos si llegamos a DCM
D = 1-Vd/Vo
#Averiguamos si estamos en CCM o DCM
Iob = Vo*Ts*((1-D)**2)*D / (2*L)
print("Iob = ", Iob, "A")
Io = Vo/R

if Io > Iob:
    #Estamos en CCM
    print("Boost CCM")
    Ix = Io/(1-D)
    #Ripple
    Delta_Il = (Vd/L) *D*Ts

    #Separamos los dos casos del calculo de C
    if Ix-Io-Delta_Il/2 >= 0:
        print("Capacitor caso 1")
        C = Io*D*Ts/Delta_Vo
    else:
        print("Capacitor caso 2")
        tx = L / (R*(1-D))
        Delta_Q = (Ix - Io + Delta_Il/2)*tx/2
        C = Delta_Q/Delta_Vo

    print("D = ", D)
    print("Io = ", Io, "A")
    print("Ix = ", Ix, "A")
    print("Delta_Il = ", Delta_Il, "A")
    print("C = ", C*1E9, "nF")

else:
    #Estamos en DCM
    print("Boost DCM")
    #Tiempos
    D = np.sqrt(( 2*L*Io / (Vd*Ts)) * (Vo/Vd - 1))
    Delta_1 = 2*L*Io / (Vd*D*Ts)
    
    #Corrientes
    Delta_Il = (Vd/L) *D*Ts
    Ix = Delta_Il*(D+Delta_1)/2

    #Separamos los dos casos del calculo de C (Funciona igual que el CCM)
    if Ix-Io-Delta_Il/2 >= 0:
        print("Capacitor caso 1")
        C = Io*D*Ts/Delta_Vo
    else:
        print("Capacitor caso 2")
        tx = L / (R*(1-D))
        Delta_Q = (Ix - Io + Delta_Il/2)*tx/2
        C = Delta_Q/Delta_Vo

    print("D = ", D)
    print("Io = ", Io, "A")
    print("Ix = ", Ix, "A")
    print("Delta_Il = ", Delta_Il, "A")
    print("C = ", C*1E9, "nF")








Iob =  0.06545454545454547 A
Boost DCM
Capacitor caso 2
D =  0.2345207879911715
Io =  0.01 A
Ix =  0.024999999999999998 A
Delta_Il =  0.12792042981336627 A
C =  7.20422134201433 nF
